In [ ]:

!pip3 install transformers requests beautifulsoup4 pandas numpy
!pip3 install torch
!pip3 install datasets
!pip3 install torch torchvision torchaudio

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
from datasets import load_dataset

ds = load_dataset("Exorde/exorde-social-media-december-2024-week1")

In [ ]:
ds

In [ ]:
## drop sentiment columns, non-English rows and get smaller sample of data
# columns_to_drop = ['sentiment', 'main_emotion','secondary_themes']
# ds['train'] = ds['train'].remove_columns(columns_to_drop)
en_ds = ds['train'].filter(lambda example: example['language'] == 'en')
sample_en_ds = en_ds.shuffle(seed=42).select(range(min(10000, len(en_ds))))
sample_en_df = sample_en_ds.to_pandas()
sample_en_df

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:

tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')
result = model(tokens)
result.logits
int(torch.argmax(result.logits))+1

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = sample_en_df['original_text']
df = pd.DataFrame(np.array(df), columns=['original_text'])
df['original_text'].iloc[0]

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['original_text'].iloc[1])

In [ ]:
df['sentiment'] = df['original_text'].apply(lambda x: sentiment_score(x[:512])) ##took 5m30s to run on 10k rows

In [ ]:
df.head()